In [7]:
from langchain_community.utilities import SQLDatabase

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

In [21]:
def connectToDB():
    db = SQLDatabase.from_uri("sqlite:///event_company_people.db")
    print(db.dialect)
    print(db.get_usable_table_names())
    print(db.get_context())
    return db

In [23]:
db=connectToDB()

sqlite
['companies', 'events', 'people']
{'table_info': '\nCREATE TABLE companies (\n\tcompany_logo_url TEXT, \n\tcompany_logo_text TEXT, \n\tcompany_name TEXT, \n\trelation_to_event TEXT, \n\tevent_url TEXT, \n\tcompany_revenue TEXT, \n\tn_employees TEXT, \n\tcompany_phone TEXT, \n\tcompany_founding_year REAL, \n\tcompany_address TEXT, \n\tcompany_industry TEXT, \n\tcompany_overview TEXT, \n\thomepage_url TEXT, \n\tlinkedin_company_url TEXT, \n\thomepage_base_url TEXT, \n\tcompany_logo_url_on_event_page TEXT, \n\tcompany_logo_match_flag TEXT, \n\tcompany_revenue_standardized TEXT, \n\tn_employees_standardized REAL, \n\tcompany_phone_cleaned REAL, \n\thas_logo INTEGER, \n\tdomain TEXT, \n\thas_linkedin INTEGER, \n\tvalid_homepage_base_url INTEGER\n)\n\n/*\n3 rows from companies table:\ncompany_logo_url\tcompany_logo_text\tcompany_name\trelation_to_event\tevent_url\tcompany_revenue\tn_employees\tcompany_phone\tcompany_founding_year\tcompany_address\tcompany_industry\tcompany_overview\th

In [25]:
table_names = db.get_usable_table_names()
table_info = ""
# for table_name in table_names:
columns = db.get_table_info(table_names)
print(columns)


CREATE TABLE companies (
	company_logo_url TEXT, 
	company_logo_text TEXT, 
	company_name TEXT, 
	relation_to_event TEXT, 
	event_url TEXT, 
	company_revenue TEXT, 
	n_employees TEXT, 
	company_phone TEXT, 
	company_founding_year REAL, 
	company_address TEXT, 
	company_industry TEXT, 
	company_overview TEXT, 
	homepage_url TEXT, 
	linkedin_company_url TEXT, 
	homepage_base_url TEXT, 
	company_logo_url_on_event_page TEXT, 
	company_logo_match_flag TEXT, 
	company_revenue_standardized TEXT, 
	n_employees_standardized REAL, 
	company_phone_cleaned REAL, 
	has_logo INTEGER, 
	domain TEXT, 
	has_linkedin INTEGER, 
	valid_homepage_base_url INTEGER
)

/*
3 rows from companies table:
company_logo_url	company_logo_text	company_name	relation_to_event	event_url	company_revenue	n_employees	company_phone	company_founding_year	company_address	company_industry	company_overview	homepage_url	linkedin_company_url	homepage_base_url	company_logo_url_on_event_page	company_logo_match_flag	company_revenue_s

In [11]:
# print(os.environ["GROQ_API_KEY"])

In [12]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192")

In [13]:
prompt = """"

"""

In [14]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "Find me companies that are attending Oil & Gas related events over the next 12 months."})
response

'SQLQuery: SELECT "company_name", "company_industry" FROM companies WHERE "company_industry" LIKE \'%oil%\' OR "company_industry" LIKE \'%gas%\' AND "event_url" IN (SELECT "event_url" FROM events WHERE "event_start_date" BETWEEN date(\'now\') AND date(\'now\', \'+1 year\') LIMIT 5);'

In [15]:
db.run('''SELECT "company_name" FROM companies WHERE "company_name" IN (SELECT "company_name" FROM companies c JOIN events e ON c."event_url" = e."event_url" WHERE date(\'now\') <= e."event_start_date" AND e."event_start_date" <= date(\'now\', \'+1 year\'));''')

'[(\'100 Women In Finance\',), (\'BBGC\',), (\'HR Maritime\',), (\'HR Maritime Consultants Ltd\',), (\'Kerry Consulting\',), (\'Marex\',), (\'Marex Inc\',), (\'Marex Solutions\',), (\'Atmospheric G2\',), (\'BIMCO\',), (\'Broadpeak\',), (\'Carbon\',), (\'Cleardox\',), (\'ClearDox LLC\',), (\'CoreTRM\',), (\'deltaconX AG\',), (\'Fenics USTreasuries\',), (\'Fenics Market Data\',), (\'Institute of Chartered Shipbrokers\',), (\'CommodityThursdays\',), (\'ITFA Americas (AMRC)\',), (\'Komgo\',), (\'Nanyang Business School\',), (\'Polestar\',), (\'POLESTAR GLOBAL LIMITED\',), (\'Propeller Club-Geneva\',), (\'SGX RegCo\',), (\'Singapore Mining Club\',), (\'Trade Data Monitor LLC\',), (\'The Broker Club\',), (\'Bloomberg\',), (\'TradeFlow Capital Management\',), (\'Value Creed\',), (\'WIMAR SG\',), (\'WISTA Singapore\',), (\'Synspective Inc.\',), (\'Airbus Defence and Space\',), (\'ICEYE\',), (\'ST Engineering North America\',), (\'Singapore Land Authority (SLA)\',), (\'IAI - Israel Aerospace In

In [16]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit


In [17]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001FC83C49790>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001FC83C49790>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001FC83C49790>),
 QuerySQLCheckerTool(description='Use this tool to 

In [18]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [19]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

C:\Users\DELL\AppData\Local\Temp\ipykernel_19680\1800557427.py:4: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)


In [20]:
example_query = "Find me companies that are attending Oil & Gas related events?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Find me companies that are attending Oil & Gas related events?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_4p2a)
 Call ID: call_4p2a
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

companies, events, people
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_nqa5)
 Call ID: call_nqa5
  Args:
    table_names: companies, events
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE companies (
	company_logo_url TEXT, 
	company_logo_text TEXT, 
	company_name TEXT, 
	relation_to_event TEXT, 
	event_url TEXT, 
	company_revenue TEXT, 
	n_employees TEXT, 
	company_phone TEXT, 
	company_founding_year REAL, 
	company_address TEXT, 
	com

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01j3y6s0z6emqsrydr2cme87jt` on tokens per minute (TPM): Limit 6000, Used 7407, Requested 13347. Please try again in 2m27.54s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:
from langchain import hub
prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

In [ ]:
prompt_template

ChatPromptTemplate(input_variables=['dialect', 'top_k'], metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'sql-agent-system-prompt', 'lc_hub_commit_hash': '31156d5fe3945188ee172151b086712d22b8c70f8f1c0505f5457594424ed352'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['dialect', 'top_k'], template='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information r